In [24]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [25]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../', '../']


In [26]:
from IPython.display import display, HTML

display(HTML('''
<style>
.output {
    display: flex;
    flex-direction: column;
}
.output_scroll {
    max-height: 300px; 
    overflow-y: auto;
}
</style>
'''))


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [27]:
x_train, y_train,x_test_original, y_test_original = ReadTS('Computers')
y_train = y_train - 1
y_test_original = y_test_original - 1
#print(x_train.shape)
#print(y_train)

In [28]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):

    original_shape = TS.shape
    

    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    

    scaler = StandardScaler()
    

    TS_normalized = scaler.fit_transform(TS_2d)

    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)

In [29]:
model = make_CNN_model(x_train.shape[1:],num_classes = 2)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=200, batch_size=32)

Epoch 1/200
7/7 [==============================] - 1s 58ms/step - loss: 0.5832 - sparse_categorical_accuracy: 0.6450 - val_loss: 0.7709 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/200
7/7 [==============================] - 0s 37ms/step - loss: 0.5420 - sparse_categorical_accuracy: 0.7050 - val_loss: 0.7341 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/200
7/7 [==============================] - 0s 37ms/step - loss: 0.5042 - sparse_categorical_accuracy: 0.7750 - val_loss: 0.7197 - val_sparse_categorical_accuracy: 0.1400
Epoch 4/200
7/7 [==============================] - 0s 37ms/step - loss: 0.4942 - sparse_categorical_accuracy: 0.7750 - val_loss: 0.6818 - val_sparse_categorical_accuracy: 0.6800
Epoch 5/200
7/7 [==============================] - 0s 34ms/step - loss: 0.4876 - sparse_categorical_accuracy: 0.7900 - val_loss: 0.6337 - val_sparse_categorical_accuracy: 1.0000
Epoch 6/200
7/7 [==============================] - 0s 35ms/step - loss: 0.4661 - sparse_categorical_ac

In [30]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

8/8 [==============================] - 0s 7ms/step - loss: 1.0840 - sparse_categorical_accuracy: 0.7640
Test accuracy 0.7639999985694885
Test loss 1.0840305089950562


### Create perturbed test sets and calculate decrease in accuracy

In [31]:
print(y_test_original.shape)

(250,)


In [32]:
print(inspect.getsource(perturb))
print(inspect.signature(perturb))

def perturb(perturbation_strategy, ts, index0, index1, global_ts = [],bg = []):
    if perturbation_strategy == 'RBP':
        return RBPIndividual(ts, index0, index1)
    if perturbation_strategy == 'zero':
        return zeroPerturb(ts, index0, index1)
    if perturbation_strategy == 'noise':
        return noisePerturb(ts, index0, index1)
    if perturbation_strategy == 'blur':
        return blurPerturb(ts, index0, index1)
    if perturbation_strategy == 'RBP1':
        return RBPIndividualNew1(global_ts, ts, index0, index1) 
    if perturbation_strategy == 'RBP1fast':
        return RBPIndividualNew1fast(bg, ts, index0, index1)  
    if perturbation_strategy == 'RBP2':
        return RBPIndividualNew2(global_ts, ts, index0, index1)    

(perturbation_strategy, ts, index0, index1, global_ts=[], bg=[])


In [51]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 500
index_1 = 720

x_test_rbp = np.asarray([perturb('RBP',x,index_0,index_1) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

pert1 = perturb('RBP1',perturb_test_set[0],index_0,index_1, global_ts = perturb_test_set)
x_test_new1 = np.asarray([perturb('RBP1',x,index_0,index_1, global_ts = perturb_test_set) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,index_0,index_1) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,index_0,index_1) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,index_0,index_1) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

8/8 [==============================] - 0s 6ms/step


In [52]:
def l2norm(perturb):
    # 计算两个数组之间的差
    difference = perturb - x_test_original
    
    # 计算差的平方
    squared_difference = np.square(difference)
    
    # 计算平方和，沿着特征维度（axis=1），因为每个时间序列样本是一行
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    # 计算平方和的平方根，即l2范数
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    # l2_norms现在是一个形状为(467,)的数组，其中每个元素代表
    # 对应时间序列样本之间的l2范数
    print(average_l2_norm)  # 输出: (467,)

l2norm(x_test_rbp)
l2norm(x_test_new1)
l2norm(x_test_zero)
l2norm(x_test_noise)
l2norm(x_test_blur)

10.882363422455764
13.849552162028115
13.829143090797025
41.972851439610835
2.0601350369371785


In [53]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [54]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [55]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.764
 RBP Accuracy: 0.648
 RBP1 Accuracy: 0.592
 Zero Accuracy: 0.596
 Noise Accuracy: 0.5
 Blur Accuracy: 0.716



In [56]:
print(np.argmax(predictions_noise,axis=1))
print(np.argmax(predictions_new1,axis=1))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0
 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0
 1 0 1 1 0 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 0
 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 0 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1